In [1]:
# All imports go here:
%matplotlib inline
import sys
import sqlite3
import pandas as pd
from pandas import DataFrame, Series
import pandas.io.sql as sql
from tabulate import tabulate

sys.path.append("../")
from LogUtils import set_frame_display


In [2]:
# function to return list of dates in ascending order:
def get_all_dates(cursor):
    """Return all dates of games in ascending order"""
    
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    dates = []
    for table in cursor.fetchall():
        table = str(table).split("'")[1]
        date = str(table).split('_')
        date = ''.join(date[-3]+'_'+date[-2]+'_'+date[-1])
        dates.append(date)
    
    return sorted(list(set(dates)))

In [3]:
def get_starters(roster):
    """
    From a roster DataFrame, returns the players in a Series that are starters
    ('Starter' column set equal to 1)
    """    
    #print roster[roster['Starter'] == 1]
    players = roster[roster['Starter'] == 1]['Name']
    #print players
    return players

def get_substitutions(gtable, column,sub_type):
    """
    Get indices where substitutions took place
    \Params:
      * gtable - table of game information
      * column - column to check where the substitution string takes place
      * sub_type - type of substitution string
    """

    valid_sub_types = ['SUB IN', 'SUB OUT']
    if sub_type not in valid_sub_types:
        raise ValueError('sub_type: %s is not valid. Please choose from: %s'
                         %(sub_type, valid_sub_types))

    return [i for i in range(gtable.shape[0]) 
            if ((gtable[column].ix[i] is not None) and
                (sub_type in gtable[column].ix[i]))]

def check_sub_errors(sub_in, sub_out):
    """
    Performs checks that substitutions have been obtained correctly.
    1) Are there any differences in sub_in/sub_out pairs greater than 5 
       (max number of players to sub at once)?
    2) Are there same number of sub in as sub out?
    """
    
    # make sure differences are all less than 5 rows - the max number of subs at any one time!
    if np.any(np.array(sub_out) - np.array(sub_in) > 5):
        raise Exception("Cannot have more than 5 subs at once! Some indexing must have gone awry")

    # Make sure there is 1 sub in for 1 sub out
    in_rows = len(sub_in)
    out_rows = len(sub_out)
    if in_rows != out_rows:
        print "in_rows: %u, out_rows: %u"%(in_rows, out_rows)
        raise Exception("Cannot have different sub_in_rows than sub_out_rows!")

    return

def make_substitutions(game_table, evt_col, sub_in, sub_out):

    for ii,idx in enumerate(sub_out):
        # Player to sub in:
        player_in = str(game_table[evt_col].ix[sub_in[ii]]).split(':')[-1][1:]
        player_out = str(game_table[evt_col].ix[idx]).split(':')[-1][1:]
        
        for col in game_table.columns:
            if game_table[col].ix[idx] == player_out:
                #print "row %d, Substituting %s for %s"%(idx,player_in, player_out)
                game_table[col][idx:] = player_in

    return

def process_play_by_play_event(game_table, side='Home'):
    """
    Processes each event in game_table and returns a new column
    with running total of score and any other statistics you'd like
    
    For scoring, as far as I know, these are the possible events:
        ['GOOD! JUMPER', 'GOOD! 3 PTR', 'GOOD! FT SHOT', 
         'GOOD! LAYUP', 'GOOD! DUNK']
    """
    
    # First question: what are all the events that can lead to a 
    # change in sore. search for 'GOOD!'?
    
    scoring_events = []
    for index in game_table[side+' Evt'].index:
        current_event = str(game_table[side+' Evt'][index])
        if 'GOOD!' in current_event:
            #print current_event
            scoring_events.append(current_event)
            if 'FT SHOT' in current_event:
                score_update = int(game_table[side+' Score'][index]) + 1
                game_table[side+' Score'][index:] = score_update
            elif '3 PTR' in current_event:
                score_update = int(game_table[side+' Score'][index]) + 3
                game_table[side+' Score'][index:] = score_update
            else:
                score_update = int(game_table[side+' Score'][index]) + 2
                game_table[side+' Score'][index:] = score_update
                
    #print set(scoring_events)
    
    return
    

def create_game_table(play_by_play, ucla_roster, opp_roster):
    """
    Creates and fills the table of 5 man unit data, the primary
    unit of information that we will process further for trends
    in roster groupings.

    \Params:
      * play_by_play - single game raw play by play DataFrame table,
        taken from sql database file.
      * ucla_roster - roster of UCLA's players to get substitution and 
        5 man unit data from.
      * opp_roster - roster of UCLA's opponents to get the subs
        and 5 man unit data from
        
    \returns
      A table (DataFrame) with columns:
        'Clock', 'Home Evt', 'Home Score', 'Away Evt', 'Away Score',
        'UCLA Player 1',...,'UCLA Player 5',
        'Opp Player 1',...,'Opp Player 5'
      where <flag> Player <#> is the player in position '#' at the
      given time stamp, and
    """
    
    # Get UCLA starters: cols=('Name', 'Jersey', 'Starter')
    ucla_starters = get_starters(ucla_roster)
    opp_starters = get_starters(opp_roster)
    game_table = DataFrame(play_by_play)
        
    #print ucla_starters
    #print opp_starters
        
    # Add columns for roster spots:
    for ii in xrange(5): game_table["UCLA Player %u"%(ii+1)] = None
    for ii in xrange(5): game_table["Opp Player %u"%(ii+1)] = None
    
    # Add columns for Home/Away score:
    game_table['Home Score'] = 0
    game_table['Away Score'] = 0
    
    # Fill in the starters to initial and all subsequent roster locations
    for ii,name in enumerate(ucla_starters):
        game_table["UCLA Player %u"%(ii+1)] = name
    for ii, name in enumerate(opp_starters):
        game_table["Opp Player %u"%(ii+1)] = name
        
        
    # Now search over Events columns to see when sub in/sub out occurs:
    sub_in_rows = get_substitutions(game_table,'Home Evt','SUB IN')
    sub_out_rows = get_substitutions(game_table,'Home Evt', 'SUB OUT')
    check_sub_errors(sub_in_rows, sub_out_rows)
    make_substitutions(game_table,'Home Evt',sub_in_rows, sub_out_rows)
    
    sub_in_rows = get_substitutions(game_table,'Away Evt','SUB IN')
    sub_out_rows = get_substitutions(game_table,'Away Evt', 'SUB OUT')
    check_sub_errors(sub_in_rows, sub_out_rows)
    make_substitutions(game_table,'Away Evt',sub_in_rows, sub_out_rows)    
    
    # Next: process events for home/away scores. This will be a generic 
    # and extensible function because I might also like to know the
    # rebounding rate or assist rate or something else while certain set
    # of guys are on the floor
    process_play_by_play_event(game_table,side='Home')
    process_play_by_play_event(game_table,side='Away')
    
    
    return game_table    

In [5]:
# Open the database and look around:
con = sqlite3.connect('../UCLA_2014_2015/UCLA_2014_2015.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Get all Tables:
all_tables = []
for table in sorted(cursor.fetchall()): 
    clean_table = str(table).split("'")[1]
    all_tables.append(clean_table)
    
all_tables = set(all_tables)

unique_dates = get_all_dates(cursor)
#print unique_dates

[0, 1, 2, 3, 4]


In [7]:
# Look at the play by play of an example game:
play_by_play = "PlayByPlay_2014_11_14"
ucla_roster = "UCLA_roster_2014_11_14"
opp_roster = "Montana_State_roster_2014_11_14"

df_play_by_play = sql.read_sql('select * from %s'%play_by_play, con)
df_ucla_roster = sql.read_sql('select * from %s'%ucla_roster, con)
df_opp_roster = sql.read_sql('select * from %s'%opp_roster, con)

df_play_by_play = df_play_by_play.drop("index",axis=1)

%time game_table = create_game_table(df_play_by_play, df_ucla_roster, df_opp_roster)


# I can't belive it, but I think this actually works!!!
for column in game_table.columns: print column
print game_table


# NEXT STEPS:
#   1) Put this into a script
#   2) Compute aggregate functions like each player's minutes, see if it matches with box score.
#   3) compute running total of home and away score by processing each home/away event - WORKING


/Users/timothyarlen/local/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/timothyarlen/local/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 4.49 s, sys: 24.9 ms, total: 4.51 s
Wall time: 4.52 s
Clock
Home Evt
Away Evt
UCLA Player 1
UCLA Player 2
UCLA Player 3
UCLA Player 4
UCLA Player 5
Opp Player 1
Opp Player 2
Opp Player 3
Opp Player 4
Opp Player 5
Home Score
Away Score
     Clock                Home Evt                   Away Evt  UCLA Player 1  \
0    19:40            MISSED LAYUP                      BLOCK  Looney, Kevon   
1    19:36                    None              REBOUND (DEF)  Looney, Kevon   
2    19:27           REBOUND (DEF)               MISSED LAYUP  Looney, Kevon   
3    19:14            MISSED 3 PTR              REBOUND (DEF)  Looney, Kevon   
4    19:04                    None               GOOD! JUMPER  Looney, Kevon   
5    18:58             GOOD! 3 PTR                       None  Looney, Kevon   
6    18:58                  ASSIST                       None  Looney, Kevon   
7    18:57                    None               MISSED 3 PTR  Looney, Kevon   
8    18:57                   

/Users/timothyarlen/local/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/timothyarlen/local/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# Now that we have the game_table - the basic unit required for processing
# Play by play and 5-man unit information, let's first compute:
#   - minutes played by each player who appears
#   - individual +/- for whoever checked in.
# Then we can proceed with the real reason for doing this-getting the 5-man unit +/-
#   Find all combinations of 5 man units, place them in a list
#   
# Define a `Lineup` class:
# class Lineup:
#   self.teamname, date, seconds played, players (set)
# 
# Can loop over game_table and where it says "Sub Out" Get the lineup at that line and 
# compute previous lineup's seconds played.


In [9]:
# Get single game and play by play for one date:
for date in unique_dates: 
    #print date
    
    # Get all tables matching the date:
    table_dates_match = []
    for table in all_tables:
        if date in table: table_dates_match.append(table)
    
    if len(table_dates_match) != 3:
        raise ValueError("tables matching date should be strictly equal to 3, "+
                         "but it's length is %u"%len(table_dates_match))
    
    # Put tables in their correct place:
    ucla_roster = ""
    opp_roster = ""
    play_by_play = ""
    for table in table_dates_match:
        if 'PlayByPlay' in table:
            play_by_play = table
        elif 'UCLA' in table:
            ucla_roster = table
        else:
            opp_roster = table
        
    print ucla_roster
    print opp_roster
    print play_by_play